# Notebook that reproduces my highest results on the public dataset (C-index of 0.7314)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
from IPython import display
import numpy.ma as ma
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import warnings 
import sksurv
from sklearn.preprocessing import MinMaxScaler
from lifelines import CoxPHFitter
from scipy.stats import spearmanr
from lifelines.utils import k_fold_cross_validation
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from metrics import cindex

In [55]:
def concat2(dfs, join='inner'):
    return pd.concat(dfs, axis=1, join=join)


def get_significant_features(X, y, source_dataset=None, multivariate=False):
    significant_features = [] 

    if multivariate:
        cph = CoxPHFitter()
        data = concat2([X, source_dataset, y])  
        cph.fit(data, duration_col='SurvivalTime', event_col='Event')

        cph.print_summary()
        
    else:
        for column in X:
            cph = CoxPHFitter()
            data = concat2([X[column], source_dataset, y])   
            cph.fit(data, duration_col='SurvivalTime', event_col='Event')

            if cph.summary.p[0] <= 0.005:
                significant_features.append([column, cph.summary.coef[0], cph.summary.z[0], '< 0.005'])
    
    df = pd.DataFrame(significant_features, columns = ['feature', 'coef', 'z', 'p'])
    
    return df

def get_features(radiomics):
    shape_cols = [col for col in radiomics.columns if 'shape' in col]
    firstorder_cols = [col for col in radiomics.columns if 'firstorder' in col]
    glcm_cols = [col for col in radiomics.columns if 'glcm' in col]
    glrlm_cols = [col for col in radiomics.columns if 'glrlm' in col]
    
    return shape_cols, firstorder_cols, glcm_cols, glrlm_cols

def cross_validate(X, y):
    all_scores = []
    
    cph = CoxPHFitter()

    for i in range(50):
        scores = k_fold_cross_validation(cph, concat2([X, y]), 'SurvivalTime', event_col='Event', k=3)
        all_scores.append(scores)

    print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(all_scores), np.std(all_scores) * 2))
    print("Max: %0.2f" % np.max(all_scores))
    print("Min: %0.2f" % np.min(all_scores))
    
    
def spearman_groups_by_mean(normalized_df, threshold=0.9, df_name=None):
    
    """
    This function averages features that are highly "Spearman-correlated" (above threshold).
    It takes care of inverting negatively correlated features before averaging, and does not re-use features
    that were previously averaged.
    """

    correlated_cols = dict()

    for column_to_test in normalized_df.columns:
        column_to_test_corrs = []
        key_already_in = None
        for column in normalized_df.columns:

            if correlated_cols.get(column) != None and column_to_test in correlated_cols.get(column):
                key_already_in = column
                continue

            if key_already_in != None and column in correlated_cols[key_already_in]:
                continue
                
            spearman_corr = spearmanr(normalized_df[column_to_test], normalized_df[column]).correlation
            
            if np.abs(spearman_corr) >= threshold:
                column_to_test_corrs.append(column)

        if key_already_in == None:        
            correlated_cols[column_to_test] = column_to_test_corrs

    
    normalized_df_grouped = pd.DataFrame()
    
    for i, (key, columns) in enumerate(correlated_cols.items()):
        name = 'group' + str(i) if df_name is None else df_name + str(i)

        if len(columns) == 1:
            normalized_df_grouped[name] = normalized_df[columns[0]]

        else:
            sum_ = 0
            for column in columns:
                if  spearmanr(normalized_df[columns[0]], normalized_df[column]).correlation < 0:
                    sum_ += 1/(normalized_df[column] + 0.0001)
                else:
                    sum_ += normalized_df[column]

                if np.isinf(sum_).sum() > 0:
                    print(columns[0], column)
                    
            mean = sum_ / len(columns)
            normalized_df_grouped[name] = mean
        
    return normalized_df_grouped, correlated_cols

# Read Data

In [56]:
def prepare_clinical(path):
    clinical = pd.read_csv(path)
    clinical = clinical.set_index('PatientID')
    clinical.sort_index(inplace=True)
    le = LabelEncoder()
    clinical['SourceDataset'] = le.fit_transform(clinical['SourceDataset'])
    return clinical

def prepare_radiomics(path):
    radiomics = pd.read_csv(path, index_col=0, header=1)[1:]
    radiomics.index = radiomics.index.astype(int)
    radiomics.sort_index(inplace=True)
    return radiomics

In [71]:
train_clinical = prepare_clinical('train/features/clinical_data.csv')
test_clinical = prepare_clinical('test/features/clinical_data.csv')

train_radiomics = prepare_radiomics('train/features/radiomics.csv')
test_radiomics = prepare_radiomics('test/features/radiomics.csv')

y_train = pd.read_csv('output.csv', index_col=0)
y_train.sort_index(inplace=True)

all_radiomics = test_radiomics.append(train_radiomics)
all_clinical = test_clinical.append(train_clinical)

# We scale the radiomics according to the whole dataset (train + test)
scaler = MinMaxScaler()
all_radiomics = pd.DataFrame(columns=all_radiomics.columns, data=scaler.fit_transform(all_radiomics)).set_index(all_radiomics.index)

train_radiomics = all_radiomics.loc[train_radiomics.index]
test_radiomics = all_radiomics.loc[test_radiomics.index]

# Get most significant features for training set

In [58]:
source_dataset = train_clinical['SourceDataset']
shape_cols, firstorder_cols, glcm_cols, glrlm_cols = get_features(train_radiomics)

significant_shape = get_significant_features(train_radiomics[shape_cols], train_y, source_dataset)
significant_firstorder = get_significant_features(train_radiomics[firstorder_cols],train_y, source_dataset)
significant_glcm = get_significant_features(train_radiomics[glcm_cols], train_y, source_dataset)
significant_glrlm = get_significant_features(train_radiomics[glrlm_cols], train_y, source_dataset)

most_significants = significant_shape.append([significant_firstorder, significant_glcm])

# Group with Spearman correlation

In [69]:
df_train_most_significants = train_radiomics[most_significants['feature']]
train_grouped_by_mean, cols_train = spearman_groups_by_mean(df_train_most_significants, 0.8)

df_test_most_significants = test_radiomics[most_significants['feature']]
test_grouped_by_mean, cols_test = spearman_groups_by_mean(df_test_most_significants, 0.8)

# Reuse groups we found in the feature selection

In [91]:
X_train = concat2([train_grouped_by_mean[['group2', 'group4']], train_clinical[['SourceDataset', 'Nstage', 'age']]])
X_train = X_train.fillna(np.mean(all_clinical.age))

X_test = concat2([test_grouped_by_mean[['group2', 'group4']], test_clinical[['SourceDataset', 'Nstage', 'age']]])
X_test = X_test.fillna(np.mean(all_clinical.age))

# Fit the model

In [92]:
cph = CoxPHFitter()
cph.fit(concat2([X_train, y_train]), duration_col='SurvivalTime', event_col='Event')
cph.print_summary()

# Make predictions on test set

In [93]:
preds = cph.predict_expectation(X_test)[0]
preds = pd.DataFrame({"SurvivalTime": preds, "Event": np.nan})
preds.head()

,SurvivalTime,Event
0,2455.537535,NaN
1,693.269017,NaN
6,975.907949,NaN
9,3057.431086,NaN
10,463.122293,NaN
